- This notebook is to get the latent space for binary mixture dataset by using different normalization(log) for weight

In [3]:
import sys
sys.path.append("../src/models")
sys.path.append("../src")
sys.path.append("../")

from utils import *
from baseline import *
from miscibility import normalize_concentrations_log

In [1]:
import pandas as pd
import numpy as np

from rdkit import Chem
from rdkit.Chem import RDKFingerprint
from rdkit.DataStructs import ConvertToNumpyArray

In [ ]:
df_mixture = pd.read_csv('../../data/Intermediate/df_mixture_20250502.csv')
# Log normalization
conc_cols = [f'conc{i}' for i in range(1, 3)]
df_mixture_norm = normalize_concentrations_log(df_mixture, conc_cols)
df_mixture_norm.head()

/Users/haoliu/anaconda3/envs/ActiveLearning/lib/python3.11/site-packages/rdkit/Chem/PandasPatcher.py:132: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  for col in df_subset.columns[df_subset.applymap(MolFormatter.is_mol).any()]


,smi1,conc1,smi2,conc2,miscibility
0,C[C@H](CCC(=O)NCCC[N+](C)(C)CCCS(=O)(=O)[O-])[...,5.097596,CC(C)(C)CC(C)(C)C1=CC=C(C=C1)OCCO,5.471754,1
1,C[C@H](CCC(=O)NCCC[N+](C)(C)CCCS(=O)(=O)[O-])[...,5.097596,[C@H]1([C@@H]([C@@H](OC([C@H]1O)C(=O)O)O[C@H]2...,3.819756,1
2,C[C@H](CCC(=O)NCCC[N+](C)(C)CCCS(=O)(=O)[O-])[...,5.097596,C([C@@H]1[C@H]([C@@H]([C@H]([C@H](O1)O[C@@H]2[...,4.116358,1
3,C[C@H](CCC(=O)NCCC[N+](C)(C)CCCS(=O)(=O)[O-])[...,5.097596,C1[C@@H]2[C@@H]([C@H](O1)[C@H]([C@H](O2)O)OS(=...,2.196230,1
4,C[C@H](CCC(=O)NCCC[N+](C)(C)CCCS(=O)(=O)[O-])[...,5.097596,C1[C@@H]2[C@@H]([C@H](O1)[C@H]([C@H](O2)O)O)O[...,1.993277,1


In [6]:
# Create separate fingerprint columns
df_mixture_norm['fp1'] = df_mixture_norm['smi1'].apply(smiles_to_fp)
df_mixture_norm['fp2'] = df_mixture_norm['smi2'].apply(smiles_to_fp)

In [10]:
X = build_feature_vector_bm(df_mixture_norm, ['fp1', 'fp2'], ['conc1', 'conc2'])
y = df_mixture_norm['miscibility'].values

In [11]:
feature_columns = [f'f_{i}' for i in range(X.shape[1])]

df_combined = pd.DataFrame(X, columns=feature_columns)

df_combined['miscibility'] = y
df_combined.head()

/Users/haoliu/anaconda3/envs/ActiveLearning/lib/python3.11/site-packages/rdkit/Chem/PandasPatcher.py:132: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  for col in df_subset.columns[df_subset.applymap(MolFormatter.is_mol).any()]


,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_2039,f_2040,f_2041,f_2042,f_2043,f_2044,f_2045,f_2046,f_2047,miscibility
0,0.000000,0.0,5.097596,0.0,0.0,0.000000,0.000000,5.097596,0.0,0.0,...,0.0,0.000000,5.097596,0.0,0.000000,0.0,0.0,5.097596,5.097596,1
1,3.819756,0.0,5.097596,0.0,0.0,3.819756,0.000000,5.097596,0.0,0.0,...,0.0,3.819756,5.097596,0.0,3.819756,0.0,0.0,5.097596,5.097596,1
2,4.116358,0.0,5.097596,0.0,0.0,4.116358,0.000000,5.097596,0.0,0.0,...,0.0,0.000000,5.097596,0.0,4.116358,0.0,0.0,5.097596,5.097596,1
3,2.196230,0.0,5.097596,0.0,0.0,2.196230,2.196230,7.293826,0.0,0.0,...,0.0,0.000000,5.097596,0.0,2.196230,0.0,0.0,7.293826,5.097596,1
4,1.993277,0.0,5.097596,0.0,0.0,1.993277,1.993277,7.090873,0.0,0.0,...,0.0,0.000000,5.097596,0.0,1.993277,0.0,0.0,5.097596,5.097596,1


In [12]:
df_combined.to_csv('df_total_binary_baseline_log_20250930.csv', index=False)